---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [1]:
import pandas as pd
import numpy as np
import re

def f1(row):
    row['Country'] = re.sub(r'[0-9]', '', row['Country'])
    
    if row['Energy Supply'] == '...':
        row['Energy Supply'] = np.NaN
    else:
        row['Energy Supply'] = row['Energy Supply'] * 1000000
    
    if row['Energy Supply per Capita'] == '...':
        row['Energy Supply per Capita'] = np.NaN
    
    if row['Country'] == 'Republic of Korea':
        row['Country'] = 'South Korea'
    
    if row['Country'] == 'United States of America':
        row['Country'] = 'United States'
    
    if row['Country'] == 'United Kingdom of Great Britain and Northern Ireland':
        row['Country'] = 'United Kingdom'
        
    if row['Country'] == 'China, Hong Kong Special Administrative Region':
        row['Country'] = 'Hong Kong'
    
    if '(' in row['Country']:
        row['Country'] = row['Country'].split('(')[0].strip()
    
    return row

def energy():
    skiprows = list(range(0,17)) + list(range(245, 400))
    energy = pd.read_excel('Energy Indicators.xls', sheetname='Energy', skiprows=skiprows, parse_cols=[2,3,4,5])
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    return energy.apply(lambda row: f1(row), axis=1)

def gdp():
    GDP = pd.read_csv('world_bank.csv', skiprows=[0,1], header=2)
    GDP.loc[GDP['Country Name'] == 'Korea, Rep.', 'Country Name'] = 'South Korea'
    GDP.loc[GDP['Country Name'] == 'Iran, Islamic Rep.', 'Country Name'] = 'Iran'
    GDP.loc[GDP['Country Name'] == 'Hong Kong SAR, China', 'Country Name'] = 'Hong Kong'
    return GDP.rename(columns={'Country Name': 'Country'})

def scimen():
    return pd.read_excel('scimagojr-3.xlsx')

def join_data():
    temp = pd.merge(energy(), gdp(), on="Country", how="inner")
    return pd.merge(temp, scimen(), on="Country", how="inner")

def answer_one():
    selecting_columns =  ['Country', 'Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    return join_data()[selecting_columns].set_index('Country').sort_values('Rank').head(15)



### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [2]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [3]:
def outer_join_data():
    temp = pd.merge(energy(), gdp(), on="Country", how="outer")
    return pd.merge(temp, scimen(), on="Country", how="outer")

def answer_two():
    return int(outer_join_data()['Country'].count() - join_data()['Country'].count())


<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [4]:
def answer_three():
    top15 = answer_one()[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    return top15.fillna(0).sum(axis=1).to_frame('avgGDP').sort_values('avgGDP', ascending=False)['avgGDP']


### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [5]:
def answer_four():
    key = answer_three().index[5]
    row = answer_one().filter(like=key, axis=0)
    return np.float64(row['2015'] - row['2006'])


### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [6]:
def answer_five():
    return answer_one()['Energy Supply per Capita'].mean()



### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [7]:
def answer_six():
    top = answer_one().sort_values('% Renewable', ascending=False)['% Renewable']
    return (top.index[0], top[0])


### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [8]:
def answer_seven():
    top15 = answer_one().apply(lambda row: row['Self-citations'] / row['Citations'], axis=1).sort_values(ascending=False)
    return (top15.index[0], top15[0])


### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [9]:
def answer_eight():
    top15 = answer_one().apply(lambda row: row['Energy Supply'] / row['Energy Supply per Capita'], axis=1).sort_values(ascending=False)
    return top15.index[2]


### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [10]:
def f9(row):
    a = row['Energy Supply'] / row['Energy Supply per Capita']
    b = row['Citable documents'] / a
    return pd.Series([b, row['Energy Supply per Capita']], index=['Citable docs per Capita', 'Energy Supply per Capita'])
    
def answer_nine():
    a = answer_one().apply(lambda row: f9(row), axis=1)
    return a.corr().iloc[0][1]


In [11]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [12]:
# plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [13]:
def f10(row, median):
    if row['% Renewable'] >= median:
        v = 1
    else:
        v = 0
    
    return v

def answer_ten():
    median = answer_one()['% Renewable'].median()
    top15 = answer_one()
    return top15.sort_values('Rank').apply(lambda row: f10(row, median), axis=1).rename('HighRenew')


### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [14]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

def f11(row, data, acc):
    data.ix[ContinentDict[row.name], 'size'] = data.ix[ContinentDict[row.name], 'size'] + 1
    data.ix[ContinentDict[row.name], 'sum'] = data.ix[ContinentDict[row.name], 'sum'] + row['PopEst']
    acc[ContinentDict[row.name]].append(row['PopEst'])

    
def answer_eleven():
    acc = {
        'Asia': [], 
        'Australia': [], 
        'Europe': [], 
        'North America': [], 
        'South America': []
    }
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    data = pd.DataFrame([], columns=['size', 'sum', 'mean', 'std'], index=['Asia', 'Australia', 'Europe', 'North America', 'South America']).fillna(0)
    data.index.name = 'Continent'
    Top15.apply(lambda row: f11(row, data, acc), axis=1)
    data.ix['Asia', 'mean'] = np.mean(acc['Asia'])
    data.ix['Australia', 'mean'] = np.mean(acc['Australia'])
    data.ix['Europe', 'mean'] = np.mean(acc['Europe'])
    data.ix['North America', 'mean'] = np.mean(acc['North America'])
    data.ix['South America', 'mean'] = np.mean(acc['South America'])
    data.ix['Asia', 'std'] = np.std(acc['Asia'])
    data.ix['Australia', 'std'] = np.std(acc['Australia'])
    data.ix['Europe', 'std'] = np.std(acc['Europe'])
    data.ix['North America', 'std'] = np.std(acc['North America'])
    data.ix['South America', 'std'] = np.std(acc['South America'])
    return data


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [33]:

def f12(row):
    row['Continent'] = ContinentDict[row.name]
    return row

def answer_twelve():
    Top15 = answer_one()
    Top15['Continent'] = ''
    Top15['Bins'] = pd.cut(Top15['% Renewable'], 5)
    r = Top15.apply(lambda row: f12(row), axis=1).groupby(['Continent', 'Bins'])
    
    return r['% Renewable'].sum()



Continent      Bins            
Asia           (15.753, 29.227]    19.754910
               (2.212, 15.753]     33.188974
Australia      (2.212, 15.753]     11.810810
Europe         (15.753, 29.227]    52.210490
               (2.212, 15.753]     10.600470
               (29.227, 42.701]    71.635820
North America  (2.212, 15.753]     11.570980
               (56.174, 69.648]    61.945430
South America  (56.174, 69.648]    69.648030
Name: % Renewable, dtype: float64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [16]:
def f13(row):
    return "{:,}".format(row['PopEst'])

def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    
    return Top15.apply(lambda row: f13(row), axis=1).rename('PopEst')


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [17]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [18]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!